## Packages Installation

First, install the `holisticai` package if you haven't already:
```bash
!pip install holisticai[all]
```
Then, import the necessary libraries.

In [1]:
import warnings

import pandas as pd
from holisticai.bias.metrics import clustering_bias_metrics
from holisticai.datasets import load_dataset
from holisticai.pipeline import Pipeline
from sklearn.cluster import KMeans

warnings.filterwarnings("ignore")

## Data Loading

In [2]:
dataset = load_dataset('clinical_records')
train_test = dataset.train_test_split(test_size=0.2, random_state=42)

train = train_test['train']
test = train_test['test']

dataset

{"dtype": "Dataset", "attributes": {"Number of Rows": 299, "Features": ["X , y , group_a , group_b"]}, "metadata": "sex: {'group_a': '0', 'group_b': '1'}"}

# 1. MCMF

#### Traditional Implementation

In [3]:
from holisticai.bias.mitigation import MCMF

# fit a clustering model
model = KMeans(n_clusters=2, random_state=42)
model.fit(train['X'])

# predict the clusters
y_pred = model.predict(train['X'])

# fit the mitigator
mitigator = MCMF(metric='L1', verbose=1)
y_pred = mitigator.fit_transform(train['X'], y_pred, train["group_a"], train['group_b'], model.cluster_centers_)['y_pred']

# get the centroids
centroids = model.cluster_centers_

# compute the bias metrics
metrics = clustering_bias_metrics(train['group_a'], train['group_b'], y_pred, data = train['X'], centroids = centroids, metric_type = 'equal_outcome')
metrics

,Value,Reference
Metric,,
Cluster Balance,0.584280,1
Minimum Cluster Ratio,0.338583,1
Cluster Distribution Total Variation,0.318793,0
Cluster Distribution KL Div,0.264755,0
Social Fairness Ratio,1.153405,1
Silhouette Difference,-0.143734,0


# Pipeline Implementation

In [4]:
mitigator = MCMF(metric='L1', verbose=1)

# set the pipeline
pipeline = Pipeline(steps=[('model', model), ('bm_postprocessing', mitigator)])
pipeline.fit(train['X'])

# predict the clusters and get the centroids
y_pred = pipeline.predict(train['X'],bm__group_a=train["group_a"],bm__group_b=train["group_b"],bm__centroids="cluster_centers_")
centroids = pipeline['model'].cluster_centers_

# compute the bias metrics
metrics_pipeline = clustering_bias_metrics(train['group_a'], train['group_b'], y_pred, data = train['X'], centroids = centroids, metric_type = 'equal_outcome')
metrics_pipeline

,Value,Reference
Metric,,
Cluster Balance,0.584280,1
Minimum Cluster Ratio,0.338583,1
Cluster Distribution Total Variation,0.318793,0
Cluster Distribution KL Div,0.264755,0
Social Fairness Ratio,1.153405,1
Silhouette Difference,-0.143734,0


### Comparison

In [5]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                           Value     Value Reference
Metric                                                              
Cluster Balance                         0.584280  0.584280         1
Minimum Cluster Ratio                   0.338583  0.338583         1
Cluster Distribution Total Variation    0.318793  0.318793         0
Cluster Distribution KL Div             0.264755  0.264755         0
Social Fairness Ratio                   1.153405  1.153405         1
Silhouette Difference                  -0.143734 -0.143734         0